# Context data for the sea level data explorer

In [82]:
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
from datetime import datetime

#get data from worldbank
from pandas_datareader import wb

import glob
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import json
import matplotlib.pyplot as plt

InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_rows = 200
#matplotlib.rcParams['svg.fonttype'] = 'none'


# Sea Level Data, Stations

In [136]:
stations = pd.read_csv('context_data/sealevel_viz_psmsl_1985_2015.csv', usecols=[2, 4, 5, 6, 7, 9])

stations = stations.drop_duplicates()

stations.rename(columns={'ID': 'id'}, inplace=True)

stations['id'].nunique()
stations.head()

513

,id,location,latitude,longitude,country,continent
0,1,Brest,48.383,-4.495,FRA,europe
31,61,Marseille,43.279,5.354,FRA,europe
62,453,Le Havre,49.482,0.106,FRA,europe
93,467,Cherbourg,49.651,-1.636,FRA,europe
124,468,Dunkerque,51.048,2.367,FRA,europe


In [110]:
stations.describe()

,id,latitude,longitude
count,513.000000,513.000000,513.000000
mean,825.904483,33.845446,25.302012
std,546.822134,27.489068,97.354383
min,1.000000,-65.246000,-177.360000
25%,368.000000,29.135000,-68.517000
50%,808.000000,38.317000,16.548000
75%,1253.000000,52.167000,129.292000
max,2316.000000,79.550000,178.423000


# Worldbank Data, Emissions

In [111]:
#get the data from worldbank, Stand: 7. April 2017

emissions = pd.read_csv('context_data/c02_emissions_worlbank.csv', usecols=[0, 1, 57])

emissions['c02_rank'] = emissions['2013'].rank(ascending=False)
emissions.rename(columns={'Country Code': 'country_code'}, inplace=True)
emissions.rename(columns={'Country Name': 'country_name'}, inplace=True)
emissions.rename(columns={'2013': 'c02_emissions'}, inplace=True)

emissions.sort_values(by='c02_rank').head()

,country_name,country_code,c02_emissions,c02_rank
198,Qatar,QAT,40.462355,1.0
240,Trinidad and Tobago,TTO,34.520237,2.0
49,Curacao,CUW,34.161635,3.0
125,Kuwait,KWT,27.258964,4.0
20,Bahrain,BHR,23.682574,5.0


# Sea Level Population Data, National Population Data

In [112]:
sealevel_pop = pd.read_csv('context_data/lecz-urban-rural-population-land-area-estimates_country-90m.csv')

In [113]:
sealevel_pop['ElevationZone'] = sealevel_pop['ElevationZone'].str.strip()
sealevel_pop['ISO3v10'] = sealevel_pop['ISO3v10'].str.strip()
pop = sealevel_pop.set_index(['ElevationZone'])
pop = pop.loc[['Elevations Less Than or Equal To 10m']]

pop = pop.reset_index()

pop_low_elevation = pop.groupby(by=['ISO3v10'])['Population2010'].sum()
pop_low_elevation = pop_low_elevation.to_frame(name = 'coastal_population2010_sum')
pop_low_elevation = pop_low_elevation.reset_index()

pop_low_elevation.head()

,ISO3v10,coastal_population2010_sum
0,ABW,21327
1,AGO,651360
2,AIA,3848
3,ALB,354132
4,ANT,46295


In [114]:
pop_total = sealevel_pop.set_index(['ElevationZone'])
pop_total = pop_total.loc[['Total National Population']]

pop_total = pop_total.reset_index()

pop_total = pop_total.groupby(by=['ISO3v10'])['Population2010'].sum()
pop_total = pop_total.to_frame(name = 'total_population2010_sum')
pop_total = pop_total.reset_index()

pop_total.head()

,ISO3v10,total_population2010_sum
0,ABW,108141
1,AGO,19618432
2,AIA,15645
3,ALB,3215988
4,ANT,202847


# Sea Level PSMSL Trend Data

In [115]:
trend_1985_2015 = pd.read_csv('context_data/trends_1985_2015.csv', usecols=[4,7])
trend_1985_2015.rename(columns={'trend': 'trend_1985_2015'}, inplace=True)
trend_1985_2015.head()

,trend_1985_2015,_id
0,2.385,1
1,3.531,7
2,1.476,8
3,1.484,9
4,2.190,10


In [53]:
pd.qcut(trend_1985_2015['trend_1985_2015'], 5)

0        (2.158, 3.131]
1        (3.131, 4.206]
2        (0.555, 2.158]
3        (0.555, 2.158]
4        (2.158, 3.131]
5        (0.555, 2.158]
6        (4.206, 9.444]
7        (2.158, 3.131]
8      [-20.451, 0.555]
9        (2.158, 3.131]
10       (2.158, 3.131]
11       (0.555, 2.158]
12       (2.158, 3.131]
13       (2.158, 3.131]
14       (2.158, 3.131]
15     [-20.451, 0.555]
16       (4.206, 9.444]
17     [-20.451, 0.555]
18       (0.555, 2.158]
19     [-20.451, 0.555]
20       (0.555, 2.158]
21       (3.131, 4.206]
22     [-20.451, 0.555]
23     [-20.451, 0.555]
24     [-20.451, 0.555]
25     [-20.451, 0.555]
26       (2.158, 3.131]
27     [-20.451, 0.555]
28     [-20.451, 0.555]
29       (2.158, 3.131]
30       (0.555, 2.158]
31       (0.555, 2.158]
32     [-20.451, 0.555]
33     [-20.451, 0.555]
34     [-20.451, 0.555]
35       (2.158, 3.131]
36       (0.555, 2.158]
37       (4.206, 9.444]
38       (0.555, 2.158]
39       (0.555, 2.158]
40       (3.131, 4.206]
41       (2.158,

In [116]:
trend_longest = pd.read_csv('context_data/trends_longest.csv', usecols=[4, 6, 12])
trend_longest.rename(columns={'trend': 'trend_longest'}, inplace=True)
trend_longest.rename(columns={'startYear': 'start_year'}, inplace=True)
trend_longest.head()

,start_year,trend_longest,_id
0,1807,0.97,1
1,1811,0.80,2
2,1968,1.73,3
3,1843,2.12,7
4,1849,1.42,8


# Risk Level Data

In [117]:
riskindex = pd.read_csv('context_data/riskindex.csv', usecols=[0,1,2,3])
riskindex.rename(columns={'Rank': 'risk_rank'}, inplace=True)
riskindex.head()

,risk_rank,Country,risk_index,risk_description
0,1,Vanuatu,36.28,very high
1,2,Tonga,29.33,very high
2,3,Philippines,26.70,very high
3,4,Guatemala,19.88,very high
4,5,Bangladesh,19.17,very high


# Create Sea Level Context Data

In [118]:
#Add emissions
sealevel_context = stations.merge(emissions, left_on='country', right_on='country_code', how='left')

sealevel_context.describe()

,id,latitude,longitude,c02_emissions,c02_rank
count,513.000000,513.000000,513.000000,500.000000,500.000000
mean,825.904483,33.845446,25.302012,10.213933,46.400000
std,546.822134,27.489068,97.354383,4.530086,42.038393
min,1.000000,-65.246000,-177.360000,0.214119,11.000000
25%,368.000000,29.135000,-68.517000,6.779624,20.000000
50%,808.000000,38.317000,16.548000,9.764393,31.000000
75%,1253.000000,52.167000,129.292000,13.532295,63.000000
max,2316.000000,79.550000,178.423000,16.389758,232.000000


In [119]:
#Add sealevel population
sealevel_context = sealevel_context.merge(pop_low_elevation, left_on='country', right_on='ISO3v10', how='left')

sealevel_context['id'].nunique()

513

In [120]:
sealevel_context.head()

,id,location,latitude,longitude,country,continent,country_name,country_code,c02_emissions,c02_rank,ISO3v10,coastal_population2010_sum
0,1,Brest,48.383,-4.495,FRA,europe,France,FRA,5.050483,92.0,FRA,2983590.0
1,61,Marseille,43.279,5.354,FRA,europe,France,FRA,5.050483,92.0,FRA,2983590.0
2,453,Le Havre,49.482,0.106,FRA,europe,France,FRA,5.050483,92.0,FRA,2983590.0
3,467,Cherbourg,49.651,-1.636,FRA,europe,France,FRA,5.050483,92.0,FRA,2983590.0
4,468,Dunkerque,51.048,2.367,FRA,europe,France,FRA,5.050483,92.0,FRA,2983590.0


In [121]:
# clean up data (remove country code)

sealevel_context = sealevel_context.drop('ISO3v10', 1)

In [122]:
#Add national population
sealevel_context = sealevel_context.merge(pop_total, left_on='country', right_on='ISO3v10', how='left')

In [123]:
sealevel_context.head()

,id,location,latitude,longitude,country,continent,country_name,country_code,c02_emissions,c02_rank,coastal_population2010_sum,ISO3v10,total_population2010_sum
0,1,Brest,48.383,-4.495,FRA,europe,France,FRA,5.050483,92.0,2983590.0,FRA,63125894.0
1,61,Marseille,43.279,5.354,FRA,europe,France,FRA,5.050483,92.0,2983590.0,FRA,63125894.0
2,453,Le Havre,49.482,0.106,FRA,europe,France,FRA,5.050483,92.0,2983590.0,FRA,63125894.0
3,467,Cherbourg,49.651,-1.636,FRA,europe,France,FRA,5.050483,92.0,2983590.0,FRA,63125894.0
4,468,Dunkerque,51.048,2.367,FRA,europe,France,FRA,5.050483,92.0,2983590.0,FRA,63125894.0


In [124]:
# clean up data (remove country code)

sealevel_context = sealevel_context.drop('ISO3v10', 1)
sealevel_context.head()

,id,location,latitude,longitude,country,continent,country_name,country_code,c02_emissions,c02_rank,coastal_population2010_sum,total_population2010_sum
0,1,Brest,48.383,-4.495,FRA,europe,France,FRA,5.050483,92.0,2983590.0,63125894.0
1,61,Marseille,43.279,5.354,FRA,europe,France,FRA,5.050483,92.0,2983590.0,63125894.0
2,453,Le Havre,49.482,0.106,FRA,europe,France,FRA,5.050483,92.0,2983590.0,63125894.0
3,467,Cherbourg,49.651,-1.636,FRA,europe,France,FRA,5.050483,92.0,2983590.0,63125894.0
4,468,Dunkerque,51.048,2.367,FRA,europe,France,FRA,5.050483,92.0,2983590.0,63125894.0


In [125]:
#Add trend data for 1985 to 2015
sealevel_context = sealevel_context.merge(trend_1985_2015, left_on='id', right_on='_id', how='left')

In [126]:
# clean up data (remove _id)

sealevel_context = sealevel_context.drop('_id', 1)

In [127]:
#Add longest trend
sealevel_context = sealevel_context.merge(trend_longest, left_on='id', right_on='_id', how='left')

In [128]:
sealevel_context.head()

,id,location,latitude,longitude,country,continent,country_name,country_code,c02_emissions,c02_rank,coastal_population2010_sum,total_population2010_sum,trend_1985_2015,start_year,trend_longest,_id
0,1,Brest,48.383,-4.495,FRA,europe,France,FRA,5.050483,92.0,2983590.0,63125894.0,2.385,1807.0,0.97,1.0
1,61,Marseille,43.279,5.354,FRA,europe,France,FRA,5.050483,92.0,2983590.0,63125894.0,3.530,1885.0,1.32,61.0
2,453,Le Havre,49.482,0.106,FRA,europe,France,FRA,5.050483,92.0,2983590.0,63125894.0,1.984,1959.0,2.66,453.0
3,467,Cherbourg,49.651,-1.636,FRA,europe,France,FRA,5.050483,92.0,2983590.0,63125894.0,1.908,1975.0,1.47,467.0
4,468,Dunkerque,51.048,2.367,FRA,europe,France,FRA,5.050483,92.0,2983590.0,63125894.0,NaN,1957.0,1.96,468.0


In [129]:
#Add riskindex
sealevel_context = sealevel_context.merge(riskindex, left_on='country_name', right_on='Country', how='left')

sealevel_context.head()

,id,location,latitude,longitude,country,continent,country_name,country_code,c02_emissions,c02_rank,coastal_population2010_sum,total_population2010_sum,trend_1985_2015,start_year,trend_longest,_id,risk_rank,Country,risk_index,risk_description
0,1,Brest,48.383,-4.495,FRA,europe,France,FRA,5.050483,92.0,2983590.0,63125894.0,2.385,1807.0,0.97,1.0,152.0,France,2.62,very low
1,61,Marseille,43.279,5.354,FRA,europe,France,FRA,5.050483,92.0,2983590.0,63125894.0,3.530,1885.0,1.32,61.0,152.0,France,2.62,very low
2,453,Le Havre,49.482,0.106,FRA,europe,France,FRA,5.050483,92.0,2983590.0,63125894.0,1.984,1959.0,2.66,453.0,152.0,France,2.62,very low
3,467,Cherbourg,49.651,-1.636,FRA,europe,France,FRA,5.050483,92.0,2983590.0,63125894.0,1.908,1975.0,1.47,467.0,152.0,France,2.62,very low
4,468,Dunkerque,51.048,2.367,FRA,europe,France,FRA,5.050483,92.0,2983590.0,63125894.0,NaN,1957.0,1.96,468.0,152.0,France,2.62,very low


In [130]:
sealevel_context.describe()

,id,latitude,longitude,c02_emissions,c02_rank,coastal_population2010_sum,total_population2010_sum,trend_1985_2015,start_year,trend_longest,_id,risk_rank,risk_index
count,513.000000,513.000000,513.000000,500.000000,500.000000,5.110000e+02,5.110000e+02,424.000000,458.000000,458.000000,458.000000,491.000000,491.000000
mean,825.904483,33.845446,25.302012,10.213933,46.400000,1.395220e+07,1.288884e+08,2.044123,1951.307860,1.376572,812.842795,104.380855,5.863198
std,546.822134,27.489068,97.354383,4.530086,42.038393,2.030402e+07,2.084985e+08,3.324252,30.451367,2.855400,548.146742,50.478684,4.256867
min,1.000000,-65.246000,-177.360000,0.214119,11.000000,3.160000e+02,2.338000e+03,-20.451000,1807.000000,-17.620000,1.000000,3.000000,1.520000
25%,368.000000,29.135000,-68.517000,6.779624,20.000000,1.300770e+06,1.139003e+07,0.965000,1938.250000,0.492500,354.750000,76.000000,3.010000
50%,808.000000,38.317000,16.548000,9.764393,31.000000,3.970153e+06,6.241743e+07,2.626000,1960.000000,1.730000,757.500000,127.000000,3.760000
75%,1253.000000,52.167000,129.292000,13.532295,63.000000,2.443121e+07,1.264972e+08,3.886500,1972.000000,2.917500,1246.750000,145.000000,6.700000
max,2316.000000,79.550000,178.423000,16.389758,232.000000,1.633811e+08,1.347565e+09,9.444000,1986.000000,8.890000,2316.000000,166.000000,26.700000


In [131]:
sealevel_context = sealevel_context.drop('Country', 1)
sealevel_context = sealevel_context.drop('country_code', 1)
sealevel_context = sealevel_context.drop('_id', 1)

In [132]:
sealevel_context.head()

,id,location,latitude,longitude,country,continent,country_name,c02_emissions,c02_rank,coastal_population2010_sum,total_population2010_sum,trend_1985_2015,start_year,trend_longest,risk_rank,risk_index,risk_description
0,1,Brest,48.383,-4.495,FRA,europe,France,5.050483,92.0,2983590.0,63125894.0,2.385,1807.0,0.97,152.0,2.62,very low
1,61,Marseille,43.279,5.354,FRA,europe,France,5.050483,92.0,2983590.0,63125894.0,3.530,1885.0,1.32,152.0,2.62,very low
2,453,Le Havre,49.482,0.106,FRA,europe,France,5.050483,92.0,2983590.0,63125894.0,1.984,1959.0,2.66,152.0,2.62,very low
3,467,Cherbourg,49.651,-1.636,FRA,europe,France,5.050483,92.0,2983590.0,63125894.0,1.908,1975.0,1.47,152.0,2.62,very low
4,468,Dunkerque,51.048,2.367,FRA,europe,France,5.050483,92.0,2983590.0,63125894.0,NaN,1957.0,1.96,152.0,2.62,very low


In [133]:
sealevel_context.describe()

,id,latitude,longitude,c02_emissions,c02_rank,coastal_population2010_sum,total_population2010_sum,trend_1985_2015,start_year,trend_longest,risk_rank,risk_index
count,513.000000,513.000000,513.000000,500.000000,500.000000,5.110000e+02,5.110000e+02,424.000000,458.000000,458.000000,491.000000,491.000000
mean,825.904483,33.845446,25.302012,10.213933,46.400000,1.395220e+07,1.288884e+08,2.044123,1951.307860,1.376572,104.380855,5.863198
std,546.822134,27.489068,97.354383,4.530086,42.038393,2.030402e+07,2.084985e+08,3.324252,30.451367,2.855400,50.478684,4.256867
min,1.000000,-65.246000,-177.360000,0.214119,11.000000,3.160000e+02,2.338000e+03,-20.451000,1807.000000,-17.620000,3.000000,1.520000
25%,368.000000,29.135000,-68.517000,6.779624,20.000000,1.300770e+06,1.139003e+07,0.965000,1938.250000,0.492500,76.000000,3.010000
50%,808.000000,38.317000,16.548000,9.764393,31.000000,3.970153e+06,6.241743e+07,2.626000,1960.000000,1.730000,127.000000,3.760000
75%,1253.000000,52.167000,129.292000,13.532295,63.000000,2.443121e+07,1.264972e+08,3.886500,1972.000000,2.917500,145.000000,6.700000
max,2316.000000,79.550000,178.423000,16.389758,232.000000,1.633811e+08,1.347565e+09,9.444000,1986.000000,8.890000,166.000000,26.700000


In [134]:
sealevel_context.to_csv('context_data/sealevel_context_data.csv')